In [1]:
## running spark locally
from pyspark import SparkContext
sc = SparkContext()

In [2]:
## pulling from mongo db
from pymongo import MongoClient

In [3]:
hashtagToPull = 'wakenbake'

In [4]:
## connect and query
client = MongoClient('mongodb://localhost:27017/')
db = client['instagram']
collectionName = hashtagToPull + '-tags'
collection = db[collectionName]
media = collection.find() ## pull entire document store

In [5]:
## load into Spark
mediaRDD = sc.parallelize(media)

In [6]:
print (mediaRDD.take(1))

[{'type': 'image', 'location': {}, 'filter': 'Normal', 'tags': ['highsociety', 'plant', 'stoner', 'stoners', 'gethigh', 'instahigh', 'marijuana', 'cannabisculture', 'staylifted', 'cannabiscommunity', 'stoned', 'weshouldsmoke', 'pothead', 'fueledbythc', 'weedstagram', 'potheadsociety', 'hightimes', 'getlifted', 'potheads', 'calistoner', 'instaweed', 'letssmoke', 'cannabis', 'stayhigh', 'dank', 'highaf', 'blowingclouds', 'wakenbake', 'highlife'], 'comment_count': 1, 'like_count': 3, 'id': '1051583761891237133_1621796574', 'caption': 'All of you :)', 'user_id': '1621796574', '_id': ObjectId('55ce3ef37795d40ecab46ad1'), 'created_time': datetime.datetime(2015, 8, 14, 18, 56, 14), 'username': 'mystical_stoner', 'comments': [{'user_id': '1621796574', 'username': 'mystical_stoner', 'text': '#Pothead #Stoner #Stoners #highaf #staylifted #cannabis #gethigh #marijuana #Calistoner #stayhigh #Weshouldsmoke #instaweed #stoned #cannabiscommunity #cannabisculture #potheadsociety #fueledbythc #getlifte

## Find most popular tags

In [18]:
allTags = (mediaRDD
           .map(lambda x: x['tags'])
           .flatMap(lambda x: x))

In [24]:
countTags = (allTags
             .map(lambda x: (x.lower(), 1))
             .reduceByKey(lambda a, b: a + b))

In [26]:
countTags.takeOrdered(20, key=lambda b: -b[1])

[('wakenbake', 49947),
 ('420', 20737),
 ('highsociety', 12122),
 ('weedstagram', 11439),
 ('highlife', 10741),
 ('cannabis', 10477),
 ('maryjane', 10003),
 ('hightimes', 9573),
 ('cannabiscommunity', 9526),
 ('stoner', 9485),
 ('dank', 9334),
 ('kush', 9103),
 ('marijuana', 9057),
 ('710', 8905),
 ('ganja', 8711),
 ('dabs', 7922),
 ('thc', 7914),
 ('weedporn', 7746),
 ('stonernation', 7108),
 ('topshelflife', 6960)]

##Number of unique users / User posts

In [32]:
uniqueUsers = (mediaRDD
               .map(lambda x: (x['username'], 1))
               .reduceByKey(lambda a, b: a + b))

In [33]:
uniqueUsers.count()

13804

In [34]:
## top users
uniqueUsers.takeOrdered(20, key=lambda b: -b[1])

[('420divas', 493),
 ('420_newsletter', 485),
 ('nochillmemecentral', 479),
 ('blazedoutlondon', 418),
 ('bobaleex', 351),
 ('maryjanebz', 311),
 ('ybcollective408', 274),
 ('hankayrec', 255),
 ('plpcsanjose', 251),
 ('just.paul.mate', 244),
 ('al_deggo', 235),
 ('officialsmokahonest', 217),
 ('50strainsofdank', 213),
 ('browneyed_toker', 205),
 ('super_star_stone', 198),
 ('ukstonerbros', 167),
 ('weednatic420', 162),
 ('amannda_420', 162),
 ('lionprideja', 160),
 ('greensocietyss', 152)]